In [2]:
# import gdown
import zipfile
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import xml.etree.ElementTree as ET
import torch
from torchvision.transforms import v2
import albumentations as A
from torch.utils.data import DataLoader, Dataset
from albumentations.pytorch import ToTensorV2
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
# from torchsummary import summary

In [4]:
class Filter():
    def __init__(self, model, transform):
        self.landmark_detect = model
        self.transform = transform
        self.face_detect = mp.solutions.face_detection.FaceDetection(min_detection_confidence=0.5, model_selection=1)
        self.landmark_detect.eval()
        
    def landmarks_detect(self, image):
        height, width, channels = image.shape
        
        image = self.transform(image=image)['image']
        in_height, in_width, in_channel = image.shape
        image = image.unsqueeze(0)
        landmarks = self.landmark_detect(image)
        landmarks = landmarks.detach().numpy()
        landmarks = landmarks.reshape(68, 2)
        
        # convert to 0-224 axis
        landmarks = (landmarks + 0.5) * in_width
        # convert to original axis
        landmarks[:, 0] = landmarks[:, 0] / in_width * width
        landmarks[:, 1] = landmarks[:, 1] / in_width * height
        
        return landmarks
        
        
    def full_landmarks_detect(self, image_path, radius=1):
        # Load the image (np.ndarray)
        image = cv2.imread(image_path)

        # Convert BGR to RGB (np.ndarray)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, channels = image_rgb.shape

        # Detect faces in the image
        faces = self.face_detect.process(image_rgb)

        # Draw rectangles around the detected faces
        for face in faces.detections:
            bbox = face.location_data.relative_bounding_box
            left = int(round(bbox.xmin * width))
            top = int(round(bbox.ymin * height))
            right = int(round((bbox.xmin + bbox.width) * width))
            bottom = int(round((bbox.ymin + bbox.height) * height))
            
            landmarks = self.landmarks_detect(image_rgb[top:bottom, left:right])
            landmarks = landmarks + [left, top]
            
            cv2.rectangle(image_rgb, (left, top), (right, bottom), (0, 255, 0), 2)
            for x, y in landmarks:
                cv2.circle(image_rgb, (int(round(x)), int(round(y))), radius=radius, color=(0, 255, 0), thickness=-1)

        # Display the image with detected faces
        plt.imshow(image_rgb)
        plt.show()

In [5]:
transform_pred = A.Compose([
    A.Resize(height=224, width=224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [6]:
filter = Filter(model, transform_pred)

AttributeError: module 'dlib' has no attribute 'get_frontal_face_detector'

In [ ]:
image_path = '/kaggle/input/test-input3/14.jpg'
filter.full_landmarks_detect(image_path, radius=1)